In [1]:
import json
import numpy as np

def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def getIDByName(labels, name):
    return labels.index(name)

def get_keypointconstants(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [float(c.strip()) for c in class_names]
    return class_names

def getHeadSize(x1,y1,x2,y2):
    headSize = 0.6*np.linalg.norm(np.subtract([x2,y2],[x1,y1]))
    return headSize

def getTopPoint(points):
    point = []
    for i in range(len(points)):
        if (points[i]["id"] is not None and points[i]["id"] == "Nose"): # if joint id matches
            point = points[i]
            break

    return point

def getPointbyID(points,id):

    point = []
    for i in range(len(points)):
        if (points[i]["id"] is not None and points[i]["id"] == id): # if joint id matches
            point = points[i]
            break

    return point

def writeJson(val,fname):
  with open(fname, 'w') as data_file:
    json.dump(val, data_file)

In [2]:
KEYPOINT_CONSTANTS = get_keypointconstants('configs/keypointconstant.txt')

def get_bb(points):
    x_values = [item["x"] for item in points]
    y_values = [item["y"] for item in points]

    # Get the minimum x and y values
    min_x = min(x_values)
    max_x = max(x_values)
    min_y = min(y_values)
    max_y = max(y_values)

    return [min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]
        
        
def compute_area(bounding_box):
    box_width = bounding_box[2][0] - bounding_box[0][0]
    box_height = bounding_box[2][1] - bounding_box[0][1]

    return box_width*box_height

def compute_scale(image_size, bounding_box):
    # Get the width and height of the image
    image_width, image_height = image_size

    # Calculate the width and height of the bounding box
    box_width = bounding_box[2][0] - bounding_box[0][0]
    box_height = bounding_box[2][1] - bounding_box[0][1]

    # Calculate the scale factors
    scale_width = box_width / image_width
    scale_height = box_height / image_height

    # Choose the minimum scale factor
    scale = min(scale_width, scale_height)

    return scale

def compute_oks(joint, d, area):
    k = KEYPOINT_CONSTANTS[joint]
    
    # Compute the exponential part of the equation
    return np.exp(-(d**2) / (2 * (area) * (k**2)))
    # The numerator expression
    numerator = np.dot(exp_vector, visibility.astype(bool).astype(int))
    # The denominator expression
    denominator = np.sum(visibility.astype(bool).astype(int))
    return numerator / denominator

In [9]:
def computeDistOks(gtLabels, gtFrames, prFrames, isImages = False):
    assert isImages or len(gtFrames) == len(prFrames)

    nJoints = len(gtLabels)
    distAll = {}
    dist = {}
    ious = {}
    iousAll = {}
    for joint in range(nJoints):
        distAll[joint] = np.zeros([0,0])
        iousAll[joint] = []

    for i, imgidx in enumerate(gtFrames):
        # ground truth
        gtFrame = gtFrames[imgidx]
        # prediction
        detFrame = prFrames[imgidx]
        prFramesLen = len(detFrame)

        if prFramesLen == 0:
            continue
        
        dist = np.ones((len(gtFrame), max(prFramesLen,1), nJoints)) * np.inf
        ious = np.zeros((len(gtFrame), max(prFramesLen,1), nJoints))
        
        for personGT in range(len(gtFrames[imgidx])):
            rectGT = gtFrames[imgidx][personGT]
            # if no poses predicted, initialise
            if prFramesLen == 0:
                for pidx in range(nJoints):
                    dist[personGT, 0, pidx] = np.inf
            else:
                # compute distance between each detection and ground truth object
                for personDT in range(prFramesLen):
                    rectPr = prFrames[imgidx][personDT]
                    if ("person" in rectGT.keys() and rectGT["person"] is not None):
                        pointsGT = rectGT["person"]["points"]
                        pointsPr = rectPr["person"]["points"]
                        bb = get_bb(pointsGT)
                        area = compute_area(bb)
                        for pidx, idxGT in enumerate(gtLabels):
                            keypointGT = getPointbyID(pointsGT,idxGT)
                            pointGT = [keypointGT["x"],keypointGT["y"]]
                            idxGT = keypointGT["id"]
                            #idx = getIDByName(gtLabels, idxGT)
                            p = getPointbyID(pointsPr,idxGT)
                            if (len(p) > 0 and
                                isinstance(p["x"], (int, float)) and
                                isinstance(p["y"], (int, float)) and 
                                p["x"] > 0 and
                                p["y"] > 0):
                                pointPr = [p["x"],p["y"]]
                                # compute distance between GT and prediction
                                d = np.linalg.norm(np.subtract(pointGT,pointPr))
                                oks = compute_oks(pidx, d, area)
                                # compute head size for distance normalization
                                head = getPointbyID(pointsGT,"Head")
                                neck = getPointbyID(pointsGT,"Neck")
                                headSize = getHeadSize(head["x"],head["y"], neck["x"],neck["y"])
                                # normalize distance
                                dNorm = d/headSize
                            else:
                                dNorm = np.inf
                                oks = 0

                            dist[personGT, personDT, pidx] = dNorm
                            ious[personGT, personDT, pidx] = oks


        #find matching detection (minimum distance) for each ground truth
        for personGT in range(len(gtFrames[imgidx])):
            min_dist = np.inf
            if prFramesLen == 0:
                minPersonDT = 0
            for personDT in range(len(prFrames[imgidx])):
                total_dist = 0
                for idx in range(nJoints):
                    if dist[personGT, personDT, idx] != np.inf:
                        total_dist += dist[personGT, personDT, idx]
                if total_dist < min_dist:
                    min_dist = total_dist
                    minPersonDT = personDT
        
            for idx in range(nJoints):
                #print(str(dist[personGT, minPersonDT, idx]) + ' ' + str(personGT) + ' ' + str(minPersonDT) + ' ' + str(idx))
                distAll[idx] = np.append(distAll[idx],[[dist[personGT, minPersonDT, idx]]])
                iousAll[idx].append(ious[personGT, minPersonDT, idx])


    return distAll, iousAll

def computeAP(iousAll, thresh):

    ap = np.zeros([len(iousAll)+1,1])
    nCorrect = 0
    nTotal = 0
    for pidx in range(len(iousAll)):
        if len(iousAll[pidx]) == 0:
            print (pidx)
            ap[pidx,0] = np.inf
        else:
            threshArr = np.ones(len(iousAll[pidx]))*thresh
            idxs = np.argwhere(iousAll[pidx] >= threshArr)
            oks = 100.0*len(idxs)/len(iousAll[pidx])
            ap[pidx,0] = oks
            nCorrect += len(idxs)
            nTotal   += len(iousAll[pidx])
    ap[len(iousAll),0] = 100.0*nCorrect/nTotal

    return ap

def computePCK(distAll,distThresh):

    pckAll = np.zeros([len(distAll)+1,1])
    nCorrect = 0
    nTotal = 0
    for pidx in range(len(distAll)):
        idxs = np.argwhere(distAll[pidx] <= distThresh)
        pck = 100.0*len(idxs)/len(distAll[pidx])
        pckAll[pidx,0] = pck
        nCorrect += len(idxs)
        nTotal   += len(distAll[pidx])
    pckAll[len(distAll),0] = 100.0*nCorrect/nTotal

    return pckAll


def computeMetrics(gtLabels, names, gtFramesAll, prFramesAll, file, isImages = False):

    distThresh = 0.5

    # compute distances
    distAll, oks = computeDistOks(gtLabels, gtFramesAll, prFramesAll, isImages)

    ap50 = computeAP(oks, distThresh)
    ap = np.copy(ap50)
    for i in range(9):
        distThresh += 0.05
        ap += computeAP(oks, distThresh)
    ap75 = computeAP(oks, 0.75)
    ap = ap/10

    # compute PCK metric
    pckAll = computePCK(distAll, distThresh)
    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': ap.flatten().tolist(),  
               'AP0.5': ap50.flatten().tolist(),  
               'AP0.75': ap75.flatten().tolist(),  
               'names': names}
    filename = 'results/results_pckh_' + file + '.json'
    print('saving results to', filename)
    writeJson(metrics,filename)

    return pckAll, ap, ap50, ap75


#load_data("test.json")

In [4]:
import os, argparse, json, re
from collections import defaultdict

def load_prediction_files(gtLabels, labels, file_path, frame_count):
    frames = defaultdict(list)
    common_labels = set()

    for filename in os.listdir(file_path):
        if filename.endswith('.json'):
            full_file = os.path.join(file_path, filename)
            with open(full_file, 'r') as f:            
                pattern = '([0-9]*)_keypoints\.json$'
                result = re.search(pattern, filename)
                frame_index = int(result.groups(0)[0])
                if frame_index < frame_count:
                    predictions_annotations = json.load(f)
                    person_id = 0
                    for pose in predictions_annotations["people"]:
                        keypoints = frames[frame_index]
                        pose_kpts = pose["pose_keypoints_2d"]
                        keypoints_index = 0
                        points = []
                        confidences = []
                        for label in labels:
                            if label in gtLabels:
                                common_labels.add(label)
                                if keypoints_index < len(pose_kpts):
                                    points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
                                    confidences.append(pose_kpts[keypoints_index+2])
                            keypoints_index = keypoints_index + 3

                        keypoint = {'points': points, 'confidence': confidences}

                        keypoints.append({'person': keypoint, 'person_id': person_id})
                        person_id += 1

                        frames[frame_index] = keypoints


    return frames, common_labels



def load_predictions(gtLabels, labels, file_path, frame_count):
  frames = defaultdict(list)
  common_labels = set()

  with open(file_path, 'r') as f:
      predictions_annotations = json.load(f)


  if ("instance_info" in predictions_annotations and predictions_annotations["instance_info"] is not None):
     for frame in predictions_annotations["instance_info"]:
        frame_index = frame["frame_id"]-1
        if frame_index < frame_count:
            person_id = 0
            keypoints = frames[frame_index]
            for pose in frame["instances"]:
                pose_kpts = pose["keypoints"]
                keypoints_index = 0
                points = []
                confidences = []
                for i, idx in enumerate(labels):
                    if idx in gtLabels:
                        common_labels.add(idx)
                    if i < len(pose_kpts):
                        points.append({'id': idx, 'x': pose_kpts[i][0], 'y': pose_kpts[i][1], 'confidence': pose["keypoint_scores"][i]})
                        confidences.append(pose["keypoint_scores"][i])

                keypoint = {'points': points, 'confidence': confidences}

                keypoints.append({'person': keypoint, 'person_id': person_id})
                person_id += 1

            frames[frame_index] = keypoints
  else:
    prev_frameindex = -1
    person_id = 0
    for pose in predictions_annotations:
        frame_index = int(pose["image_id"].replace(".jpg", ""))
        if prev_frameindex == frame_index:
            person_id += 1
        else:
            person_id = 0
        if frame_index < frame_count:
            keypoints = frames[frame_index]
            pose_kpts = pose["keypoints"]
            keypoints_index = 0
            points = []
            confidences = []
            for label in labels:
                if keypoints_index < len(pose_kpts):
                    points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
                    confidences.append(pose_kpts[keypoints_index+2])
                    keypoints_index = keypoints_index + 3

            keypoint = {'points': points, 'confidence': confidences}

            keypoints.append({'person': keypoint, 'person_id': person_id })

            frames[frame_index] = keypoints
        prev_frameindex = frame_index

  return frames, common_labels

def load_annotations(labels, file_path):
    frames = defaultdict(list)
    isExist = os.path.exists(file_path)
    if isExist:
      annotations = []
      with open(file_path, 'r') as f:
          annotations = json.load(f)

          framecount = annotations['image']['frame_count']
          keypoints = defaultdict(list)
          for person in annotations['annotations']:
            
            for frame_index in range(0, framecount):
              points = []
              if frame_index < len(person['frames']):
                frame = person['frames'][str(frame_index)]
                for node in frame['skeleton']['nodes']:
                  if node['name'] in labels:
                    points.append({'id': node['name'], 'x' : node['x'], 'y': node['y']})


              if len(points) > 0:
                keypoints[frame_index].append({'person': {'points': points}})

          for frame in range(0, len(keypoints)):
            frames[frame] = keypoints[frame]

    return frames

def load_data(file):
    with open(file, 'r') as f:
      manifest = json.load(f)

    gtLabels = get_classes('configs/halpe26_classes.txt') 
    apAll = np.zeros([len(gtLabels)+1,1])
    ap50All = np.zeros([len(gtLabels)+1,1])
    ap75All = np.zeros([len(gtLabels)+1,1])
    pckAll = np.zeros([len(gtLabels)+1,1])
    
    names = gtLabels[:]
    names.append('Total')
    for i in manifest['index']:
        print('Processing ' + i['file'])

        gt = load_annotations(gtLabels, i['annotations'])
        labels = get_classes(manifest['labels'])
        predictions = i['predictions']
        if os.path.isfile(predictions):
            pred, labels = load_predictions(gtLabels, labels, predictions, len(gt))
        else:
            pred, labels = load_prediction_files(gtLabels, labels, predictions, len(gt))

        pck, ap, ap50, ap75 = computeMetrics(gtLabels, names, gt, pred, manifest['model'] + '_' + i['name'])
        pckAll = (pckAll + pck)
        apAll = (apAll + ap)
        ap50All = (ap50All + ap50)
        ap75All = (ap75All + ap75)

    pckAll = pckAll/len(manifest['index'])
    apAll = apAll/len(manifest['index'])
    ap50All = ap50All/len(manifest['index'])
    ap75All = ap75All/len(manifest['index'])
    
    metrics = {'pckh': pckAll.flatten().tolist(), 
               'AP': apAll.flatten().tolist(),  
               'AP0.5': ap50All.flatten().tolist(),  
               'AP0.75': ap75All.flatten().tolist(),  
               'names': names}
    filename = 'results/results_' + manifest['model'] + '.json'
    print('saving results to', filename)
    writeJson(metrics,filename)

In [14]:

load_data("vitpose.json")
load_data("rtmpose_cocowb.json")
load_data("openpose.json")
load_data("alphapose_halpe.json")

Processing clip_marathon_1.mp4
saving results to results/results_pckh_vitpose_clip_marathon_1.json
Processing clip_tri_1.mp4
saving results to results/results_pckh_vitpose_clip_tri_1.json
Processing clip_tri_3.mp4
saving results to results/results_pckh_vitpose_clip_tri_3.json
Processing clip_10k_2.mp4
saving results to results/results_pckh_vitpose_clip_10k_2.json
Processing short.mp4
saving results to results/results_pckh_vitpose_short.json
saving results to results/results_vitpose.json
Processing clip_tri_1.mp4
saving results to results/results_pckh_rtmpose_clip_tri_1.json
Processing clip_tri_3.mp4
saving results to results/results_pckh_rtmpose_clip_tri_3.json
Processing clip_marathon_1.mp4
saving results to results/results_pckh_rtmpose_clip_marathon_1.json
Processing clip_10k_2.mp4
saving results to results/results_pckh_rtmpose_clip_10k_2.json
Processing short.mp4
saving results to results/results_pckh_rtmpose_short.json
saving results to results/results_rtmpose.json
Processing clip_

In [5]:
def load_mpii_predictions(gtLabels, labels, file_path):
    frames = defaultdict(list)
    common_labels = set()

    for filename in os.listdir(file_path):
        if filename.endswith('.json'):
            full_file = os.path.join(file_path, filename)

            

In [6]:

def load_mpii_annotations(labels, file_path):
    frames = defaultdict(list)
    isExist = os.path.exists(file_path)
    if isExist:
      annotations = []
      with open(file_path, 'r') as f:
        annotations = json.load(f)

        for image in annotations['annotations']:
            points = []
            pose_kpts = image["keypoints"]
            frame = image['image_id']
            id = image['id']
            keypoints_index = 0
            for label in labels:
                if keypoints_index < len(pose_kpts):
                    points.append({'id': label, 'x': pose_kpts[keypoints_index], 'y': pose_kpts[keypoints_index+1], 'confidence': pose_kpts[keypoints_index+2]})
                    keypoints_index = keypoints_index + 3

            if len(points) > 0:
                frames[frame].append({'person': {'id': id, 'points': points}})

    return frames

In [17]:
def load_mpii_data(file):
    with open(file, 'r') as f:
      manifest = json.load(f)

    gtLabels = get_classes('configs/mpii_classes.txt')
    gt = load_mpii_annotations(gtLabels, manifest['annotations'])
    names = gtLabels[:]
    names.append('Total')
    cnt = 0
    for i in manifest['model']:
        labels = get_classes(i['labels'])
        predictionsFolder = i['predictions']
        predictionsFile = i['file']
        model = i['model']
        allPredictions = defaultdict(list)
        if model == 'openpose':
           allPredictions = load_prediction_files(gtLabels, labels, predictionsFolder, np.inf)
        else:
            for filename in os.listdir(predictionsFolder):
                fileDir = os.path.join(predictionsFolder, filename)
                if os.path.isdir(fileDir):
                    if filename.isdigit():
                        fileDir = os.path.join(fileDir,  predictionsFile)
                        pred, prLabels = load_predictions(gtLabels, labels, fileDir, np.inf)
                        allPredictions = merge_defaultdicts(allPredictions, pred)
                        cnt += 1

        pck, ap, ap50, ap75 = computeMetrics(gtLabels, names, gt, allPredictions, 'mpii_' + model, True)
        
        metrics = {'pckh': pck.flatten().tolist(), 
                'AP': ap.flatten().tolist(),  
                'AP0.5': ap50.flatten().tolist(),  
                'AP0.75': ap75.flatten().tolist(),  
                'names': names}
        resFile = 'results/results_mpii_' + model + '.json'
        print('saving results to', resFile)
        writeJson(metrics, resFile)

    return

def merge_defaultdicts(d,d1):
    for k,v in d1.items():
        if (k in d):
            d[k].update(d1[k])
        else:
            d[k] = d1[k]
    return d

In [18]:
load_mpii_data('mpii.json')

saving results to results/results_pckh_mpii_alphapose.json
saving results to results/results_mpii_alphapose.json
